En caso de que alguien más esté atascado en el ejercicio final, dejo a continuación un código de ejemplo relacionado con el ejercicio de follow up de virtual dataset. Recordad que hacer un virtual dataset no es solo incluir la variable sobre la que hago la agregación (tas y/o time), sino que hay que incluir todas las variables de los ficheros originales y sus atributos. El fichero final tiene que poder abrirse con xarray, si no se puede está incompleto y mal realizado.

Un saludo.
```python
src1 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_201501-210012.nc")
src2 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_210101-230012.nc")
f = h5py.File("VDS.h5", "w")

# bnds dimension
f.create_dataset("bnds", (2,), "f8")
f["bnds"].make_scale(b"This is a netCDF dimension but not a netCDF variable.         2")

# lat coordinate
layout = h5py.VirtualLayout(shape=src1["lat"].shape, dtype=src1["lat"].dtype)
layout[...] = h5py.VirtualSource(src1.filename, "lat", shape=src1["lat"].shape)
f.create_virtual_dataset("lat", layout)
f["lat"].make_scale("lat")
f["lat"].attrs.create("units", src1["lat"].attrs["units"])

# lat_bnds
layout = h5py.VirtualLayout(shape=src1["lat_bnds"].shape, dtype=src1["lat_bnds"].dtype)
layout[...] = h5py.VirtualSource(src1.filename, "lat_bnds", shape=src1["lat_bnds"].shape)
f.create_virtual_dataset("lat_bnds", layout)

# time coordinate (need to perform join existing)
joined_time = src1["time"].shape[0] + src2["time"].shape[0]
layout = h5py.VirtualLayout(shape=(joined_time,), dtype=src1["time"].dtype)
end_time = src1["time"].shape[0]
layout[0:end_time] = h5py.VirtualSource(src1.filename, "time", shape=src1["time"].shape)
layout[end_time:] = h5py.VirtualSource(src2.filename, "time", shape=src2["time"].shape)
f.create_virtual_dataset("time", layout)
f["time"].make_scale("time")
f["time"].attrs.create("units", src1["time"].attrs["units"])
f["time"].attrs.create("calendar", src1["time"].attrs["calendar"])

# time_bnds (also remember the join existing)
layout = h5py.VirtualLayout(shape=(joined_time,2), dtype=src1["time_bnds"].dtype)
layout[0:end_time] = h5py.VirtualSource(src1.filename, "time_bnds", shape=src1["time_bnds"].shape)
layout[end_time:] = h5py.VirtualSource(src2.filename, "time_bnds", shape=src2["time_bnds"].shape)
f.create_virtual_dataset("time_bnds", layout)
f.close()
# Close the source files
src1.close()
src2.close()
```

Adjunto codigo que puede ser util para hacer la practica final.
```python
with h5py.File("project.nc", "w") as f:
   f["variant_label"] = ["r1i1p1f1", "r2i1p1f1"]
   f["variant_label"].attrs.create("standard_name", "realization", dtype="S11")
   f["variant_label"].make_scale("variant_label")
   f.create_virtual_dataset("bnds", h5py.VirtualLayout(shape=(2,), dtype='i4'))
   f["bnds"].make_scale("This is a netCDF dimension but not a netCDF variable. 2")
   tas_layout = h5py.VirtualLayout((1,2,3,4))ç
   tas_layout[0,0:end,...,...] = tas_2015_r1
   tas_layout[0,end:,...,...] = tas_2101_r1 
   tas_layout[1,0:end,...,...] = tas_2015_r2
   tas_layout[1,end:,...,...] = tas_2101_r2
   ```

funciones:
hdf5
f.create_dataset
f.make_scale
f.dims.attach_scale

VD
h5py.VirtualLayout
h5py.VirtualSource
f.create_virtual_dataset
f.attrs.create

In [ ]:
#prueba 1
tas_sim1_2015 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_201501-210012.nc")
tas_sim1_2101 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_210101-230012.nc")
tas_sim2_2015 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_201501-210012.nc")
tas_sim2_2101 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_210101-230012.nc")
pr_sim1_2015 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_201501-210012.nc")
pr_sim1_2101 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_210101-230012.nc")
pr_sim2_2015 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_201501-210012.nc")
pr_sim2_2101 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_210101-230012.nc")


with h5py.File("project.nc", "w") as f:

    #creamos virtual datasets
    #dataset time
    joined_time = tas_sim1_2015["time"].shape[0] + tas_sim1_2101["time"].shape[0]
    layout = h5py.VirtualLayout(shape=(joined_time,), dtype=tas_sim1_2015["time"].dtype)
    end = tas_sim1_2015["time"].shape[0]
    layout[0:end] = h5py.VirtualSource(tas_sim1_2015.filename, "time", tas_sim1_2015["time"].shape)
    layout[end:] = h5py.VirtualSource(tas_sim1_2101.filename, "time", tas_sim1_2101["time"].shape) 
    f.create_virtual_dataset("time", layout)
    for atributo in dict(tas_sim1_2015["time"].attrs).keys():
        f["time"].attrs.create(atributo, tas_sim1_2015["time"].attrs[atributo])
    
    #dataset lat
    lenlat = tas_sim1_2015["lat"].shape[0] 
    layout = h5py.VirtualLayout(shape=(lenlat,), dtype=tas_sim1_2015["lat"].dtype)
    layout[0:lenlat] = h5py.VirtualSource(tas_sim1_2015.filename, "lat", tas_sim1_2015["lat"].shape)
    f.create_virtual_dataset("lat", layout)
    for atributo in dict(tas_sim1_2015["lat"].attrs).keys():
        f["lat"].attrs.create(atributo, tas_sim1_2015["lat"].attrs[atributo])

    #dataset lon
    lenlon = tas_sim1_2015["lon"].shape[0] 
    layout = h5py.VirtualLayout(shape=(lenlon,), dtype=tas_sim1_2015["lon"].dtype)
    layout[0:lenlon] = h5py.VirtualSource(tas_sim1_2015.filename, "lon", tas_sim1_2015["lon"].shape)
    f.create_virtual_dataset("lon", layout)
    for atributo in dict(tas_sim1_2015["lon"].attrs).keys():
        f["lon"].attrs.create(atributo, tas_sim1_2015["lon"].attrs[atributo])
    
    #dataset bnds
    lenbnds = tas_sim1_2015["bnds"].shape[0] 
    layout = h5py.VirtualLayout(shape=(lenbnds,), dtype=tas_sim1_2015["bnds"].dtype)
    layout[0:lenbnds] = h5py.VirtualSource(tas_sim1_2015.filename, "bnds", tas_sim1_2015["bnds"].shape)
    f.create_virtual_dataset("bnds", layout)
    for atributo in dict(tas_sim1_2015["bnds"].attrs).keys():
        f["bnds"].attrs.create(atributo, tas_sim1_2015["bnds"].attrs[atributo])
        
    #dataset variant_label
    f["variant_label"] = ["r1i1p1f1", "r2i1p1f1"]
    f["variant_label"].attrs.create("standard_name", b"realization", dtype="S11")

    # f.create_dataset(name="variant_label",shape=(2,))
    # f["variant_label"][()] = np.array([1,2])
    # f["variant_label"].attrs.create("standard_name", b"realization", dtype="S11")
    # f.create_dataset("variant_label", data=[1, 2], dtype='i')

    #datset height
    layout = h5py.VirtualLayout(shape=(), dtype=tas_sim1_2015["height"].dtype)
    layout[()] = h5py.VirtualSource(tas_sim1_2015.filename, "height", tas_sim1_2015["height"].shape)
    f.create_virtual_dataset("height", layout)
    for atributo in dict(tas_sim1_2015["height"].attrs).keys():
        f["height"].attrs.create(atributo, tas_sim1_2015["height"].attrs[atributo])
        
    #dataset lon_bnds
    dim1= tas_sim1_2015["lon"].shape[0] 
    dim2=tas_sim1_2015["bnds"].shape[0] 
    layout = h5py.VirtualLayout(shape=(dim1,dim2), dtype=tas_sim1_2015["lon_bnds"].dtype)
    layout[()] = h5py.VirtualSource(tas_sim1_2015.filename, "lon_bnds", tas_sim1_2015["lon_bnds"].shape)
    f.create_virtual_dataset("lon_bnds", layout)
    for atributo in dict(tas_sim1_2015["lon_bnds"].attrs).keys():
        f["lon_bnds"].attrs.create(atributo, tas_sim1_2015["lon_bnds"].attrs[atributo])
        
    #dataset lat_bnds
    dim1= tas_sim1_2015["lat"].shape[0] 
    dim2=tas_sim1_2015["bnds"].shape[0] 
    layout = h5py.VirtualLayout(shape=(dim1,dim2), dtype=tas_sim1_2015["lat_bnds"].dtype)
    layout[()] = h5py.VirtualSource(tas_sim1_2015.filename, "lat_bnds", tas_sim1_2015["lat_bnds"].shape)
    f.create_virtual_dataset("lat_bnds", layout)
    for atributo in dict(tas_sim1_2015["lat_bnds"].attrs).keys():
        f["lat_bnds"].attrs.create(atributo, tas_sim1_2015["lat_bnds"].attrs[atributo])

    
    #dataset time_bnds
    dim1= tas_sim1_2015["time"].shape[0] 
    dim2=tas_sim1_2015["bnds"].shape[0] 
    layout = h5py.VirtualLayout(shape=(dim1,dim2), dtype=tas_sim1_2015["time_bnds"].dtype)
    layout[()] = h5py.VirtualSource(tas_sim1_2015.filename, "time_bnds", tas_sim1_2015["time_bnds"].shape)
    f.create_virtual_dataset("time_bnds", layout)
    for atributo in dict(tas_sim1_2015["time_bnds"].attrs).keys():
        f["time_bnds"].attrs.create(atributo, tas_sim1_2015["time_bnds"].attrs[atributo])

    
    #dataset tas
    lenvar = f['variant_label'].shape[0]
    layout = h5py.VirtualLayout(shape=(lenvar,joined_time,lenlat,lenlon), dtype=tas_sim1_2015["tas"].dtype)
    end = tas_sim1_2015["time"].shape[0]
    layout[0,0:end] = h5py.VirtualSource(tas_sim1_2015.filename, "tas", tas_sim1_2015["tas"].shape)
    layout[0,end:] = h5py.VirtualSource(tas_sim1_2101.filename, "tas", tas_sim1_2101["tas"].shape)
    layout[1,0:end] = h5py.VirtualSource(tas_sim2_2015.filename, "tas", tas_sim2_2015["tas"].shape)
    layout[1,end:] = h5py.VirtualSource(tas_sim2_2101.filename, "tas", tas_sim2_2101["tas"].shape)
    f.create_virtual_dataset("tas", layout)
    for atributo in dict(tas_sim1_2015["tas"].attrs).keys():
        f["tas"].attrs.create(atributo, tas_sim1_2015["tas"].attrs[atributo])
        
    #dataset pr
    layout = h5py.VirtualLayout(shape=(lenvar,joined_time,lenlat,lenlon), dtype=pr_sim1_2015["pr"].dtype)
    layout[0,0:end] = h5py.VirtualSource(pr_sim1_2015.filename, "pr", pr_sim1_2015["pr"].shape)
    layout[0,end:] = h5py.VirtualSource(pr_sim1_2101.filename, "pr", pr_sim1_2101["pr"].shape)
    layout[1,0:end] = h5py.VirtualSource(pr_sim2_2015.filename, "pr", pr_sim2_2015["pr"].shape)
    layout[1,end:] = h5py.VirtualSource(pr_sim2_2101.filename, "pr", pr_sim2_2101["pr"].shape)
    f.create_virtual_dataset("pr", layout)
    for atributo in dict(pr_sim1_2015["pr"].attrs).keys():
        f["pr"].attrs.create(atributo, pr_sim1_2015["pr"].attrs[atributo])
    


    # #creamos dimension scales
    # #tas
    # f["variant_label"].make_scale("variant_label")
    # f["time"].make_scale("time")
    # f["lat"].make_scale("lat")
    # f["lon"].make_scale("lon")
    
    # f["tas"].dims[0].attach_scale(f["variant_label"])
    # f["tas"].dims[1].attach_scale(f["time"])
    # f["tas"].dims[2].attach_scale(f["lat"])
    # f["tas"].dims[3].attach_scale(f["lon"])
    # #pr
    # f["pr"].dims[0].attach_scale(f["variant_label"])
    # f["pr"].dims[1].attach_scale(f["time"])
    # f["pr"].dims[2].attach_scale(f["lat"])
    # f["pr"].dims[3].attach_scale(f["lon"])
    
    # #time_bnds
    # f["bnds"].make_scale("bnds")
    # f["time_bnds"].dims[0].attach_scale(f["time"])
    # f["time_bnds"].dims[1].attach_scale(f["bnds"])
    
    # #lat_bnds
    # f["lat_bnds"].dims[0].attach_scale(f["lat"])
    # f["lat_bnds"].dims[1].attach_scale(f["bnds"])
    
    # #lon_bnds
    # f["lon_bnds"].dims[0].attach_scale(f["lon"])
    # f["lon_bnds"].dims[1].attach_scale(f["bnds"])

    
tas_sim1_2015.close()
tas_sim1_2101.close()
tas_sim2_2015.close()
tas_sim2_2101.close()
pr_sim1_2015.close()
pr_sim1_2101.close()
pr_sim2_2015.close()
pr_sim2_2101.close()

In [ ]:
#notas
tas_sim1_2015 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_201501-210012.nc")
tas_sim1_2101 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_210101-230012.nc")
tas_sim2_2015 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_201501-210012.nc")
tas_sim2_2101 = h5py.File("tas_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_210101-230012.nc")
pr_sim1_2015 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_201501-210012.nc")
pr_sim1_2101 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r1i1p1f1_gn_210101-230012.nc")
pr_sim2_2015 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_201501-210012.nc")
pr_sim2_2101 = h5py.File("pr_Amon_ACCESS-ESM1-5_ssp585_r2i1p1f1_gn_210101-230012.nc")

#print(dict(src["lat"].attrs).keys())

f = h5py.File("project.nc", "w")




# bnds dimension
f.create_dataset("bnds", (2,), "f8")
f["bnds"].make_scale(b"This is a netCDF dimension but not a netCDF variable.         2")

# lat coordinate
layout = h5py.VirtualLayout(shape=src1["lat"].shape, dtype=src1["lat"].dtype)
layout[...] = h5py.VirtualSource(src1.filename, "lat", shape=src1["lat"].shape)
f.create_virtual_dataset("lat", layout)
f["lat"].make_scale("lat")
f["lat"].attrs.create("units", src1["lat"].attrs["units"])

# lat_bnds
layout = h5py.VirtualLayout(shape=src1["lat_bnds"].shape, dtype=src1["lat_bnds"].dtype)
layout[...] = h5py.VirtualSource(src1.filename, "lat_bnds", shape=src1["lat_bnds"].shape)
f.create_virtual_dataset("lat_bnds", layout)

# time coordinate (need to perform join existing)
joined_time = src1["time"].shape[0] + src2["time"].shape[0]
layout = h5py.VirtualLayout(shape=(joined_time,), dtype=src1["time"].dtype)
end_time = src1["time"].shape[0]
layout[0:end_time] = h5py.VirtualSource(src1.filename, "time", shape=src1["time"].shape)
layout[end_time:] = h5py.VirtualSource(src2.filename, "time", shape=src2["time"].shape)
f.create_virtual_dataset("time", layout)
f["time"].make_scale("time")
f["time"].attrs.create("units", src1["time"].attrs["units"])
f["time"].attrs.create("calendar", src1["time"].attrs["calendar"])

# time_bnds (also remember the join existing)
layout = h5py.VirtualLayout(shape=(joined_time,2), dtype=src1["time_bnds"].dtype)
layout[0:end_time] = h5py.VirtualSource(src1.filename, "time_bnds", shape=src1["time_bnds"].shape)
layout[end_time:] = h5py.VirtualSource(src2.filename, "time_bnds", shape=src2["time_bnds"].shape)
f.create_virtual_dataset("time_bnds", layout)


f.close()
# Close the source files
tas_sim1_2015.close()
tas_sim1_2101.close()
tas_sim2_2015.close()
tas_sim2_2101.close()
pr_sim1_2015.close()
pr_sim1_2101.close()
pr_sim2_2015.close()
pr_sim2_2101.close()

In [ ]:
with h5py.File("project.nc", "w") as f:
    f["variant_label"] = ["r1i1p1f1", "r2i1p1f1"]
    f["variant_label"].attrs.create("standard_name", b"realization", dtype="S11")
    f["variant_label"].make_scale("variant_label")
    f.create_virtual_dataset("bnds", h5py.VirtualLayout(shape=(2,), dtype='i4'))
    f["bnds"].make_scale("This is a netCDF dimension but not a netCDF variable.         2")

    # tas_layout = h5py.VirtualLayout((1,2,3,4))
    # tas_layout[0,0:end,...,...] = tas_2015_r1
    # tas_layout[0,end:,...,...] = tas_2101_r1 
    # tas_layout[1,0:end,...,...] = tas_2015_r2
    # tas_layout[1,end:,...,...] = tas_2101_r2